# Зависимости

In [ ]:
import csv
import json
import random

from core.settings import (FUNCTIONS_PATH, 
                           FILTERED_FUNCTIONS_PATH, 
                           DATASET_PATH,
                           UNIQ_NAMES_PATH,
                           UNIQ_VALUES_PATH)
from core.dataprocess import (read_portion_data, 
                              append_functions_in_json, 
                              filter_functions_name, 
                              dataset_save_to_csv, 
                              load_functions_from_json,
                              timer,
                              count_file_lines,
                              get_uniq_dictcounters,
                              dictcounter_to_json,
                              dictcounter_from_json,
                              cut_dictcounter,
                              filter_functions)

# Выполнение анализа имен функций

### Подсчёт количества функций

Ондна функция занимает одну строку в файле. Поэтому необходимо посчитать количество строк в файле с функциями.

In [ ]:
funcs_num = count_file_lines(FUNCTIONS_PATH)

print('Общее количество функций в датасете:', funcs_num)

### Подсчет уникальных имен функций и значений узлов

* `unames` - словарь-счетчик уникальных имен функций

* `uvalues` - словарь-счетчик уникальных значений узлов функций

Словарь-счетчик:
    
`dict -> {item: amount, item2: amount2, ...}`

In [ ]:
unames, uvalues = get_uniq_dictcounters(FUNCTIONS_PATH)

print(f'Уникальных имен функций: {len(unames)}\n' \
        f'Уникальных значений узлов: {len(uvalues)}')

**Импорт/экспорт уникальных значений и имен функций в/из файл(а) json.**

In [ ]:
# экспорт
dictcounter_to_json(unames, UNIQ_NAMES_PATH)
dictcounter_to_json(uvalues, UNIQ_VALUES_PATH)

In [ ]:
# импорт
unames = dictcounter_from_json(UNIQ_NAMES_PATH)
uvalues = dictcounter_from_json(UNIQ_VALUES_PATH)

print(f'Уникальных имен функций: {len(unames)}\n' \
        f'Уникальных значений узлов: {len(uvalues)}')

### Выбор самых частотных имен функций

`*_th` - (thresholder) задает порог вхождения имен функций или значений в выборку. Если имен или значений больше порога, то такое имя или значение входит в выборку, а которые не входят отбрасываются.

In [ ]:
name_th = 400
value_th = 250

unames = cut_dictcounter(unames, name_th)
uvalues = cut_dictcounter(uvalues, value_th)

print(f'Уникальных имен функций, после сокращения: {len(unames)}\n' \
        f'Уникальных значений узлов, после сокращения: {len(uvalues)}')

### Фильтрация имен функций

**Плохие имена**

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [ ]:
bad_names = ['teardown', 'check', 'shutdown', 'size', 'cleanup', 'main', 'run', 'init', 'iter', 'enter', 'hash', 'start', 'nonzero', 'func', 'foo', '_run', '__call__', '__enter__', '__exit__', '__new__', 'func', 'fn', 'cb', 'g', 'backwards', 'forwards']
unames = set(filter(
    lambda name: name not in bad_names and len(name) > 2, unames
))

In [ ]:
print('Уникальных имен функций после фильтрации: ', len(unames))

### Фильтрация уникальных значений

Избавление от значений не несущих много информации

In [ ]:
#uvalues = set(filter(lambda value: len(value) > 1, uvalues)) # ?

In [ ]:
print('Уникальных значений после фильтрации: ', len(uvalues))

## Фильтрация функций по выбранным именам и трансформация узлов

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

### Фильтрация по имени

Сохранение отфильтрованных функций json файл в (`FILTERED_FUNCTIONS_PATH`).

In [ ]:
ff_num = filter_functions(FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH, unames)

print('Количество функций после фильтрации:', ff_num)

### Трансформация узлов

Загрузка отфильтрованных функций из json файла (`FILTERED_FUNCTIONS_PATH`).

In [ ]:
functions = load_functions_from_json(FILTERED_FUNCTIONS_PATH)

print('Количество функций:', len(functions))

In [ ]:
random.shuffle(functions)

#### Извлечение названия функции из функции

Удаление из обучающей выборки названия функции и сохранение в отдельном массиве для дальнейшей валидации.

In [ ]:
# Проверка, что все первые узлы последовательностей
# являются объявлением функции
for func in functions:
    assert func[0]['type'] == 'FunctionDef'

In [ ]:
labels = []
for i, func in enumerate(functions):
    labels.append(functions[i][0]['value'])
    del functions[i][0]

#### Трансформация

In [ ]:
for i,function in enumerate(functions):
    for j,node in enumerate(function):
        try:
            functions[i][j] = node['value'] if node['value'] in uvalues else node['type']
        except KeyError:
            functions[i][j] = node['type']
            
        bad_symbols = ['\n', "'", '"', '\r', '\0']
        for bs in bad_symbols:
            functions[i][j] = functions[i][j].replace(bs, ' ')

In [ ]:
# Проверка, что у каждой последовательности есть лэйбл
assert len(labels) == len(functions)

In [ ]:
print('Пример лэйбла и последовательности:')
print(labels[0])
print(functions[0])

In [ ]:
ave = sum(len(func) for func in functions) / len(functions)
print('Средняя длина вектора АСД: ' + str(ave))

**Схожая логика функции**

Объединение названий меток функций. Используется для функций со схожей логикой работы.

In [ ]:
labels_converter = {
    'getitem': 'get',
    'getattr': 'get',
    'setitem': 'set',
    'setattr': 'set',
    'unicode': 'str',
    'repr': 'str',
    'delete': 'remove',
    'delitem': 'remove',
    'test_init': 'test',
    'test_simple': 'test',
    'setupclass': 'setup',
    'teardownclass': 'teardown',
    'contains': 'exists',
}
for i, label in enumerate(labels):
    if label in labels_converter:
        labels[i] = labels_converter[label]

# Сохранение финального датасета

Соединение функции с ее меткой (названием)

In [ ]:
functions_dataset = [(label, func) for (label, func) in zip(labels, functions)]

Сохранение финального датасета для обучения в `.csv` файл

In [ ]:
dataset_save_to_csv(DATASET_PATH, functions_dataset)

In [ ]:
#end